# Auswertung der Fragebogen

In [1]:
import json
import pathlib

import numpy as np
import pandas as pd

In [2]:
path = pathlib.Path("pilot")

In [3]:
def survey_data(directory):
    for proband in directory.glob("*"):
        pseudonym = proband.name
        data = list()
        for file in pathlib.Path(proband, "fragebogen-daten").glob("*.json"):
            with file.open("r", encoding="utf-8") as survey:
                answers = pd.Series(json.load(survey))
                # Save name:
                answers.name = file.stem
                if "nasa" in answers.name or "quesi" in answers.name:
                    # Drop the website row:
                    answers = answers.drop("website")
                    # Need integers:
                    answers = answers.apply(int)
                data.append(answers)
        yield {"pseudonym": pseudonym, "data": data}

In [4]:
K = {"item1", "item6", "item11"}
Z = {"item2", "item7", "item12"}
L = {"item3", "item8", "item13"}
V = {"item4", "item9", "item14"}
Fe = {"item5", "item10"}

In [5]:
def aggregate_survey(data, website):
    tlx = list()
    quesi = list()
    pre = list()
    post = list()
    for proband in data:
        pseudonym = proband["pseudonym"]
        data = proband["data"]
        for survey in data:
            if survey.name == f"{website}-nasa":
                # Multiply values (between 1 and 20) with 5,
                # sum and divide by number of items (6)
                score = survey * 5
                score = score.sum() / 6
                tlx.append(score)
            elif survey.name == f"{website}-quesi":
                # Add 1 to values (need 1 to 5, but have 0 to 4)
                survey = survey + 1
                score = list()
                k = list()
                z = list()
                l = list()
                v = list()
                fe = list()
                for item, value in zip(survey.index, survey):
                    if item in K:
                        k.append(value)
                    elif item in Z:
                        z.append(value)
                    elif item in L:
                        l.append(value)
                    elif item in V:
                        v.append(value)
                    elif item in Fe:
                        fe.append(value)
                score.append(pd.Series(k).median())
                score.append(pd.Series(z).median())
                score.append(pd.Series(l).median())
                score.append(pd.Series(v).median())
                score.append(pd.Series(fe).median())
                quesi.append(pd.Series(score).mean())
            elif survey.name == "pre":
                survey.name = pseudonym
                pre.append(survey)
            elif survey.name == "post":
                survey.name = pseudonym
                post.append(survey)
    return pd.Series(tlx), pd.Series(quesi), pd.DataFrame(pre), pd.DataFrame(post)

In [43]:
data = survey_data(path)
tlx, quesi, pre, post = aggregate_survey(data, "spreadshirt")

## logs

In [7]:
def logfile_data(directory):
    for proband in directory.glob("*"):
        pseudonym = proband.name
        data = list()
        for file in pathlib.Path(proband, "logfile-daten").glob("*.log"):
            logs = pd.read_csv(file, sep=";", header=None)
            # Save name:
            logs.name = file.stem
            data.append(logs)
        yield {"pseudonym": pseudonym, "data": data}

In [58]:
x = list(logfile_data(path))

In [9]:
def aggergate_logfiles(data, website):
    feedback = list()
    keyboard = list()
    mouse = list()
    for proband in data:
        pseudonym = proband["pseudonym"]
        data = proband["data"]
        for logfile in data:
            if logfile.name == f"{website}-feedback":
                counts = logfile[1].value_counts()
                counts.name = pseudonym
                feedback.append(counts)
            elif logfile.name == f"{website}-keyboard":
                keyboard.append(logfile[0].count())
            elif logfile.name == f"{website}-mouse":
                mouse.append(logfile[0].count())
    return pd.DataFrame(feedback).fillna(0), pd.Series(keyboard), pd.Series(mouse)

In [115]:
feedback, keyboard, mouse = aggergate_logfiles(x, "spreadshirt")

# Auswertung

parameter zusammenrechnen?? oder einfach mehrere tests?
https://machinelearningmastery.com/statistical-power-and-power-analysis-in-python/

In [21]:
data_surv = survey_data(path)
data_logf = logfile_data(path)

tlx_spread, quesi_spread, pre_spread, post_spread = aggregate_survey(data_surv, "spreadshirt")
feedback_spread, keyboard_spread, mouse_spread = aggergate_logfiles(data_logf, "spreadshirt")

data_surv = survey_data(path)
data_logf = logfile_data(path)

tlx_shirt, quesi_shirt, pre_shirt, post_shirt = aggregate_survey(data_surv, "shirtinator")
feedback_shirt, keyboard_shirt, mouse_shirt = aggergate_logfiles(data_logf, "shirtinator")

## Effizienz

1. Klicks/Tastatur
2. NASA

? Mausklicks gewichten, weil die wichtiger sind ?

In [35]:
print("TLX:", tlx_spread.mean(), tlx_spread.std())
print("Klicks:", mouse_spread.mean(), mouse_spread.std())
print("Tastatur:", keyboard_spread.mean(), keyboard_spread.std())

TLX: 10.8333333333 12.9636243218
Klicks: 106.0 5.65685424949
Tastatur: 30.5 2.12132034356


In [36]:
print("TLX:", tlx_shirt.mean(), tlx_shirt.std())
print("Klicks:", mouse_shirt.mean(), mouse_shirt.std())
print("Tastatur:", keyboard_shirt.mean(), keyboard_shirt.std())

TLX: 16.6666666667 7.07106781187
Klicks: 91.0 36.7695526217
Tastatur: 27.5 0.707106781187


In [43]:
from numpy import std, mean, sqrt

#correct if the population S.D. is expected to be equal for the two groups.
def cohen_d(x=tlx_spread,y=tlx_shirt):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)

cohen_d()

-0.55866081912733545

In [75]:
from statsmodels.stats.power import TTestIndPower

samples = list()
e = list()

for values in [(tlx_shirt, tlx_spread), (mouse_shirt, mouse_spread), (keyboard_shirt, keyboard_spread),
               (quesi_spread, quesi_shirt), (feedback_spread["pos"], feedback_shirt["pos"]),
               (feedback_spread["neg"], feedback_shirt["neg"])]:
    a = values[0]
    b = values[1]
    effect = cohen_d(a, b)
    e.append(effect)
    alpha=0.15
    power=0.7
    # perform power analysis
    analysis = TTestIndPower()
    result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)
    #print('Sample Size: %.3f' % result)
    samples.append(result)
print(pd.Series(samples).median())

25.3018977198


In [53]:
quesi_spread

0    2.3
1    4.1
dtype: float64

In [52]:
quesi_shirt

0    2.4
1    4.1
dtype: float64

## Effektivität

Nicht messbar

## Zufriedenheit

1. QUESI
2. Feedback

In [32]:
print("QUESI:", quesi_spread.mean())
print("Positiv:", feedback_spread["pos"].mean())
print("Negativ:", feedback_spread["neg"].mean())

QUESI: 3.2
Positiv: 0.5
Negativ: 4.5


In [33]:
print("QUESI:", quesi_shirt.mean())
print("Positiv:", feedback_shirt["pos"].mean())
print("Negativ:", feedback_shirt["neg"].mean())

QUESI: 3.25
Positiv: 1.0
Negativ: 3.0


## Stichprobengröße mit GPower

In [34]:
pd.Series([10.8333333333, 16.6666666667]).std()

4.1247895569686666